In [1]:
%matplotlib inline

In [2]:
import numpy as np
from collections import deque
import pandas as pd
from matplotlib import pyplot as plt
from tqdm import tqdm_notebook as tqdm

* The Seasonal Autoregressive model $(p, 0, 0)\times(P,0,0)_s$ assumes the time series is generated according to the following model

$$ \phi(B) \Phi(B^s) X_t = \epsilon_t, \tag{1} \quad \{\epsilon_t\} \sim WN(0, \sigma^2), \label{eq:1}$$

where

$ \phi(B) = 1 - \overline{\phi}(B), $

$ \Phi(B) = 1 - \overline{\Phi}(B^s), $

$ \overline{\phi}(B) = \phi_1 B + \phi_2 B^2 + \ldots + \phi_p B^p , $

$ \overline{\Phi}(B^s) = \Phi_1 B^s + \Phi_2 B^{2s} + \ldots + \Phi_P B^{Ps} , $

* From equation $\eqref{eq:1}$, $X_t$ is generated via the formula

$$ X_t = \overline{\phi}(B) X_t + \overline{\Phi}(B^s) X_t - \overline{\phi}(B)\overline{\Phi}(B^s) X_t + \epsilon_t $$

* Prediction at time $t$ is calculated as

$$\tilde{X}_t = \overline{\phi}(B) X_t + \overline{\Phi}(B^s) X_t - \overline{\phi}(B)\overline{\Phi}(B^s) X_t$$

* Squared loss function is defined as follows

\begin{align}
f_t(\phi, \Phi) &= \ell_t(X_t, \tilde{X}_t (\phi, \Phi)) \\
                &= (X_t - \tilde{X}_t (\phi, \Phi))^2 \\
                &= \epsilon_t^2 \\
                &= [\phi(B) \Phi(B^s) X_t]^2
\end{align}

* Gradients of the loss function with respect to $\phi$ and $\Phi$ are calculated as

$$ \frac{\partial f_t}{\partial \phi_i} = -2 (X_t - \tilde{X}_t) B^i \Phi(B^s) X_t, \quad i=\overline{1,p} $$
$$ \frac{\partial f_t}{\partial \Phi_j} = -2 (X_t - \tilde{X}_t) B^{js} \phi(B) X_t, \quad j=\overline{1,P} $$
$$ \nabla_t = \nabla f_t(\phi, \Phi) = (\frac{\partial f_t}{\partial \phi_1}, \ldots, \frac{\partial f_t}{\partial \phi_p}, \frac{\partial f_t}{\partial \Phi_1}, \ldots, \frac{\partial f_t}{\partial \Phi_P})^T $$

* Seasonal ARMA Online Newton Step algorithm:

Input: non-seasonal order $p$, seasonal order $P$, seasonal period $s$.

Set 

$c = 1,$ 

$D = 2c \cdot \sqrt{p + P},$ 

$G = D \cdot X_{max}^2,$

$\lambda = \frac{1}{p+P},$

$\eta = \frac{1}{2} \min{(4GD, \lambda)},$

$\epsilon = \frac{1}{(\eta D)^2},$

and an initial matrix $A_0$ $(p + P) \times (p + P)$

$A_0 = \epsilon I_{p+P}.$

Choose $(\phi, \Phi)^1 \in K = \{ \gamma \in \mathbb{R}^{p+P}, | \gamma_j | \leq c, j = 1,\ldots,p+P \}$ arbitrarily.

for $t=1$ to $T-1$ do

&emsp; Predict $\tilde{X}_t = \overline{\phi}(B) X_t + \overline{\Phi}(B^s) X_t - \overline{\phi}(B)\overline{\Phi}(B^s) X_t$

&emsp; Observe $X_t$ and compute loss $f_t((\phi, \Phi)^t)$

&emsp; Let $\nabla_t = \nabla f_t((\phi, \Phi)^t)$, update $A_t \leftarrow A_{t-1} + \nabla_t \nabla_t^T$ 

&emsp; Set $(\phi, \Phi)^{t+1} \leftarrow \Pi^{A_t}_K ((\phi, \Phi)^{t} - \frac{1}{\eta} A_t^{-1} \nabla_t)$

end for

In [3]:
class SARMA_ONS:
    '''
    This class implements Seasonal ARMA Online Newton Step algorithm
    '''
    def __init__(self, history, order, order_s, period):
        self.order = order # AR order p
        self.order_s = order_s # Seasonal AR order P
        self.period = period # Seasonal period s
        
        # [X_{t-1}, X_{t-2}, ..., X_{t-(p+s*P)}]
        self.hist_window_reversed = np.zeros(order + period * order_s)
        trunc = list(reversed(history))[:order + period * order_s]
        self.hist_window_reversed[:len(trunc)] = trunc
        self.hist_window_reversed = deque(self.hist_window_reversed, maxlen=order + period * order_s)
        
        self.c = 1
        self.X_max = 2
        self.D = 2 * self.c * np.sqrt(order + order_s)
        self.G = self.D * (self.X_max**2)
        self.lambda_ = 1.0 / (order + order_s)
        self.eta = 0.5 * min(4 * self.G * self.D, self.lambda_) # learning rate
        self.epsilon = 1.0 / (self.eta * self.D)**2
        self.A = np.matrix(np.diag([1] * (order + order_s)) * self.epsilon) # hessian matrix
        self.gamma = np.matrix(np.random.uniform(-self.c, self.c, (order, 1))) # parameters g_1, g_2, ..., g_p
        self.gamma_s = np.matrix(np.random.uniform(-self.c, self.c, (order_s, 1))) # seasonal parameters gs_1, gs_2, ..., gs_P
        
        
    def _compute_backshift(self, polycoef):
        '''
        Compute a_1 * X_{t-1} + a_2 * X_{t-2} + ... + a_k * X_{t-k}
        
        inputs:
         polycoef = [a_0, a_1, a_2, ..., a_k]
         self.hist_window_reversed = [X_{t-1}, X_{t-2}, ..., X_{t-(p+s*P)}]
        '''
        return np.dot(polycoef, [0] + list(self.hist_window_reversed)[:len(polycoef)-1])
        
        
    def predict(self, gamma_polycoef, gamma_s_polycoef):
        '''
        X_pred_t = gamma_polynomial(B) X_t + gamma_s_polynomial(B^s) X_t - gamma_polynomial(B) * gamma_s_polynomial(B^s) X_t
        where
         gamma_polynomial(B) = gamma_1 * B + gamma_2 * B^2 + ... + gamma_p * B^p
         gamma_s_polynomial(B^s) = gamma_s_1 * B^s + gamma_s_2 * B^(2s) + ... + gamma_s_P * B^(Ps)
         
        params:
         gamma_polycoef = [gamma_1, gamma_2, ..., gamma_p]
         gamma_s_polycoef = [[0]*(s-1), gamma_s_1, [0]*(s-1), gamma_s_2, ... , [0]*(s-1), gamma_s_P]
        '''
        gamma_polycoef = [0] + gamma_polycoef
        gamma_s_polycoef = [0] + gamma_s_polycoef
        
        X_pred1 = self._compute_backshift(gamma_polycoef)
        X_pred2 = self._compute_backshift(gamma_s_polycoef)
        
        poly_multiply = np.polymul(np.flip(gamma_polycoef), np.flip(gamma_s_polycoef))
        poly_multiply = np.flip(poly_multiply)
        X_pred3 = self._compute_backshift(poly_multiply)
        return X_pred1 + X_pred2 - X_pred3
    
    
    def update_parameters(self, x, x_pred, gamma_polycoef, gamma_s_polycoef):
        '''
        delta(loss_t)/delta(gamma_i) = -2 * (X_t - X_pred_t) * [B^i * gamma_s_polynomial(B^s)](X_t), i=1,...,p
        delta(loss_t)/delta(gamma_s_j) = -2 * (X_t - X_pred_t) * [B^(js) * gamma_polynomial(B)](X_t), j=1,...,P
        where
         gamma_polynomial(B) = 1 - gamma_1 * B - gamma_2 * B^2 - ... - gamma_p * B^p
         gamma_s_polynomial(B^s) = 1 - gamma_s_1 * B^s - gamma_s_2 * B^(2s) - ... - gamma_s_P * B^(Ps)
        
        params:
         gamma_polycoef = [gamma_1, gamma_2, ..., gamma_p]
         gamma_s_polycoef = [[0]*(s-1), gamma_s_1, [0]*(s-1), gamma_s_2, ... , [0]*(s-1), gamma_s_P]
        '''
        gamma_polycoef = [1] + list(-np.array(gamma_polycoef))
        gamma_s_polycoef = [1] + list(-np.array(gamma_s_polycoef))
        
        # compute gradient w.r.t gamma and gamma_s
        polynomial_list = [[0]*i + gamma_s_polycoef for i in range(1, self.order+1)]
        nabla = [self._compute_backshift(polynomial) for polynomial in polynomial_list]
        
        polynomial_s_list = [[0]*(i*self.period) + gamma_polycoef for i in range(1, self.order_s+1)]
        nabla_s = [self._compute_backshift(polynomial_s) for polynomial_s in polynomial_s_list]
        
        nabla_all = -2 * (x - x_pred) * np.array(nabla + nabla_s)
        
        # reshape
        nabla_all = nabla_all.reshape(-1,1)
        
        # update parameters
        self.A += np.dot(nabla_all, nabla_all.T)
        grad = 1 / self.eta * np.dot(np.linalg.inv(self.A), nabla_all)
        self.gamma -= grad[:self.order]
        self.gamma_s -= grad[self.order:]
    
    
    def update_history(self, x):
        self.hist_window_reversed.appendleft(x)
        
    
    def fit_one_step(self, x):
        '''
        Run one iteration of the algorithm
        
        params:
         x: float, observation value at t (X_t)
        
        returns:
         x_pred: float, prediction value at t (Xpred_t)
         loss: float, squared loss (x - x_pred)^2
        '''
        gamma_polycoef = list(np.array(self.gamma).squeeze(-1)) # [g_1, ..., g_p]
        gamma_s_polycoef = list(np.array(self.gamma_s).squeeze(-1)) # [gs_1, ..., gs_P]
        gamma_s_polycoef = list(np.kron(gamma_s_polycoef, [0]*(self.period-1) + [1])) # [[0]*(s-1), gs_1, [0]*(s-1), gs_2, ..., gs_P]
        
        # predict
        x_pred = self.predict(gamma_polycoef, gamma_s_polycoef)
        
        # compute loss
        loss = (x - x_pred)**2
        
        # update parameters
        self.update_parameters(x, x_pred, gamma_polycoef, gamma_s_polycoef)
        
        # update history
        self.update_history(x)
        
        return x_pred, loss

In [4]:
df = pd.read_csv('data/Dữ liệu phụ tải workingday miền Bắc năm 2015-082019.csv', 
                 usecols=list(map(str,range(1,25))), engine='python')
MAPEs = []

In [5]:
for i in tqdm(range(1,25)):
    hour = str(i)
    series = df[hour]
    series_diff = series.diff().dropna()
    split_point = round(len(series) * 0.8)
    
    # normalize data
    mean = series_diff.mean()
    std = series_diff.std()
    series_diff_normalized = (series_diff - mean) / std
    
    # train
    model = SARMA_ONS([], order=3, order_s=2, period=250)

    preds_diff_normalized = []
    for x in series_diff_normalized:
        x_pred, loss = model.fit_one_step(x)
        preds_diff_normalized.append(x_pred)
        
    # forecast
    preds_diff = np.array(preds_diff_normalized) * std + mean
    preds = np.r_[[0], series[:-1].values + np.array(preds_diff)]
    forecast = pd.Series(preds, index=series.index)[split_point:]

    testing = series[split_point:]
    
    MAPEs.append(np.mean(np.abs((testing - forecast) / testing)))

In [6]:
print(MAPEs)
print(np.mean(MAPEs))

[0.05679872860679685, 0.05502600219300754, 0.052509710637228535, 0.050109220669971234, 0.04683890509217262, 0.039710899356154065, 0.03443508317149593, 0.03632499415947579, 0.039822858516609175, 0.04360560007381473, 0.04697241075810682, 0.05303817625300357, 0.056524624640247746, 0.06564048568659717, 0.04891321943308923, 0.0402084665604697, 0.03287686187023267, 0.029562950610582013, 0.03396986222370003, 0.037224901399611994, 0.04330325472493238, 0.051434378419175406, 0.05706941336000028, 0.053607462771670514]
0.04606368629950608
